Using Beatiful Soup library to Scrap Companies' information Pages from ZaubaCrop Website

In [31]:
from urllib.request import urlopen
import csv
from csv import writer
from bs4 import BeautifulSoup as soup
import re

Converting "Biofuel Companies Home Page" into BeautifulSoup format

In [32]:
url='https://www.zaubacorp.com/companysearchresults/Biofuel-'
uClient=urlopen(url)
page_html=uClient.read()
page_soup=soup(page_html)

Now Getting all the hyperlinks to internal Pages and saving it as a list

In [33]:
def getLinks(page_html, includeUrl):
    Links=[]
    for link in page_html.findAll("a", href=re.compile("^(/|.*"+includeUrl+")")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in Links:
                Links.append(link.attrs['href'])
    return Links

def splitAddress(address):
    addressParts = address.replace("http://", "").split("/")
    return addressParts

company_links= getLinks(page_soup, splitAddress(url)[0])

Refining the List to include only Companies' information Page link

In [38]:
del company_links[0:14]
del company_links[339:367]
#company_links

Creating Database of Relevant Information about the Company from each page's table 

In [36]:
def Database_creation(company_link):
    uClient=urlopen(company_link);
    page_soup=soup(uClient);
    table = page_soup.findAll("table",{"class":"table table-striped"})[0]
    rows = table.findAll("tr")
    i=0;
    csvRow_L=[]
    for row in rows:
        for cell in row.findAll(['td']):
            if i/2!=0:
                csvRow_L.append(cell.get_text());
                i+=1;
            else:
                i+=1;
                continue;
        i=0;
    return(csvRow_L)

def Database_head(company_link):
    uClient=urlopen(company_link);
    page_soup=soup(uClient);
    table = page_soup.findAll("table",{"class":"table table-striped"})[0]
    rows = table.findAll("tr")
    i=0;
    csvRow_L=[]
    for row in rows:
        for cell in row.findAll(['td']):
            if i/2==0:
                csvRow_L.append(cell.get_text());
                i+=1;
            else:
                i+=1;
                continue;
        i=0;
    return(csvRow_L)
    

Now Running the Loop for filling of Database with all the information saved as lists then converted to csv file

In [37]:
csvRow_G=[]
for company in company_links:
    if company_links.index(company)==0:
        csvRow_G.append(Database_head(company));
    csvRow_G.append(Database_creation(company));
    
with open("/Users/yashchauhan/Desktop/final2.csv", "wt") as f:
    writer = csv.writer(f)
    writer.writerows(csvRow_G)
